In [2]:
#import the libraries as needed
from selenium import webdriver
from selenium import webdriver   # for webdriver
from selenium.webdriver.support.ui import WebDriverWait  # for implicit and explict waits
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np
import json
from bs4 import BeautifulSoup
import requests

from tqdm.notebook import tqdm # to create loadbard in for loop
import os
from datetime import date
import time
import random
import pickle
import pandas as pd
from datetime import datetime

For now dowload all earning call transcript for a specified ticker

### url to use for apple
https://seekingalpha.com/api/v3/symbols/aapl/transcripts?filter[until]=undefined&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]=20

https://seekingalpha.com/api/v3/symbols/aapl/transcripts?filter[until]=1501632962&id=aapl&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=false&page[size]=20

In [2]:
def create_url_for_scrap(ticker, publish_time_min ):
    # accept publis_time_min = None
    return_per_request  = 20 #the one used by seekingalpha by default
    ticker = ticker.lower()
    publish_time_min = "undefined" if publish_time_min ==None else str(publish_time_min)
    to_return = ("https://seekingalpha.com/api/v3/symbols/"+ticker+"/transcripts?filter[until]="+publish_time_min
                 +"&id="+ticker
                 +"&include=author%2CprimaryTickers%2CsecondaryTickers%2Csentiments&isMounting=true&page[size]="
                 + str(return_per_request))
    return to_return
def is_earning_call(element_response):
    title = element_response["attributes"]["title"].lower()
    has_earning_in_title = "earning" in title or "earnings" in title
    has_call_in_title = "call" in title
    return element_response["type"]=="transcript" and has_call_in_title and has_earning_in_title
def get_url_id_date_earning_call(element_response):
    date = element_response["attributes"]["publishOn"][:10]
    call_id = element_response["id"]
    url = element_response["links"]["self"]
    return url, call_id, date

In [3]:
def get_sp500_tickers():
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    df_tickers = table[0]
    tickers=df_tickers["Symbol"].values
    tickers= tickers.tolist()
    return tickers

def get_sp600_tickers():
    table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_600_companies#S&P_600_Component_Stocks')
    df_tickers = table[1]
    tickers = df_tickers['Ticker symbol'].values
    tickers = tickers.tolist()
    return tickers

## Parameters

In [4]:
# CELL TO CHANGE
executable_path = './chromedriver'
ticker_html_path = "data/sp600/ticker/"
tickers = get_sp600_tickers() 
tickers = tickers[:]
print(tickers)

['AAN', 'AAON', 'AAT', 'AAWW', 'ABCB', 'ABG', 'ABM', 'ABTX', 'ACA', 'ACLS', 'ADC', 'ADTN', 'ADUS', 'AEIS', 'AEL', 'AGO', 'AGYS', 'AHH', 'AIN', 'AIR', 'AIT', 'AJRD', 'AKR', 'ALEX', 'ALG', 'ALGT', 'ALRM', 'AMBC', 'AMCX', 'AMEH', 'AMN', 'AMPH', 'AMSF', 'AMWD', 'ANDE', 'ANF', 'ANGO', 'ANIK', 'ANIP', 'AORT', 'AOSL', 'APEI', 'APOG', 'ARCB', 'ARI', 'ARLO', 'ARNC', 'AROC', 'ARR', 'ASIX', 'ASO', 'ASTE', 'ATEN', 'ATGE', 'ATI', 'ATNI', 'AVA', 'AVAV', 'AVD', 'AVNS', 'AWR', 'AX', 'AXL', 'AZZ', 'B', 'BANC', 'BANF', 'BANR', 'BBBY', 'BCC', 'BCOR', 'BCPC', 'BDN', 'BFS', 'BGS', 'BHE', 'BHLB', 'BIG', 'BJRI', 'BKE', 'BKU', 'BLFS', 'BLMN', 'BMI', 'BOOM', 'BOOT', 'BRC', 'BRKL', 'BSIG', 'CAKE', 'CAL', 'CALM', 'CAMP', 'CARA', 'CARS', 'CASH', 'CATO', 'CBU', 'CCOI', 'CCRN', 'CCS', 'CCSI', 'CDMO', 'CEIX', 'CELH', 'CENT', 'CENTA', 'CENX', 'CEVA', 'CFFN', 'CHCO', 'CHCT', 'CHEF', 'CHRS', 'CHS', 'CHUY', 'CIR', 'CIVI', 'CLB', 'CLDT', 'CLW', 'CMP', 'CMTL', 'CNK', 'CNMD', 'CNSL', 'CNXN', 'COHU', 'COKE', 'COLB', 'COLL',

In [5]:
# for now scrapp all earning call transcript from one company
for ticker in tqdm(tickers):
    html_path = ticker_html_path + ticker ## todo save file with name time step*
    last_time_scrap_path = html_path+ "last_time_scrap"
    if not os.path.exists(html_path):
        os.makedirs(html_path)
        print(f"The new directory {html_path} has been created!")
        
    if os.path.exists(last_time_scrap_path):
        with open(last_time_scrap_path, "r") as file:
            publish_time_min = file.read()
    else : 
        publish_time_min = None
        
    if publish_time_min !="all":
        more_data = True
        service = Service(executable_path)
        option = webdriver.ChromeOptions()
        driver = webdriver.Chrome(service=service,options=option)
        id_URLs = []
        id_to_date = {}
        while more_data:
            url = create_url_for_scrap(ticker, publish_time_min)
            driver.get(url)
            json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
            data_responses = json.loads(json_response)
            if "meta" not in data_responses:
                time.sleep(600) # 2 chances to be flag as robot
                driver.get(url)
                json_response = driver.find_element(by=By.TAG_NAME, value = "body").text
                data_responses = json.loads(json_response)
                if "meta" not in data_responses:
                    last_publish_time_min = publish_time_min
                    break
            publish_time_min = str(data_responses["meta"]["page"]["minmaxPublishOn"]["min"]) # need to be string
            if publish_time_min!= 'None':
                for element in data_responses["data"]:
                    if is_earning_call(element):
                        URL, call_id, date = get_url_id_date_earning_call(element)
                        id_URLs.append((call_id,URL))
                        id_to_date[call_id] = date
            else :
                last_publish_time_min = "all"
                more_data = False
            time.sleep(2+random.uniform(0, 1))
            
        # Download HTML files
        domain_name = 'https://seekingalpha.com'
        for call_id, URL in tqdm(id_URLs):
            file_path = html_path + "/"+ call_id + ".html"
            with open(file_path, "w") as file :
                url = domain_name+URL
                html =  requests.get(url).text
                file.write(html)

        # Save the last scrapped url of the ticker, so that if bot detected can resume
        with open(html_path+"/last_time_scrap", "w") as file:
            file.write(last_publish_time_min)    

        # Check last file got downloaded, i.e. captcha/bot error
        #BeautifulSoup(html).get_text()[:2000]

        # Save date of transcripts
        id_to_date_path = ticker_html_path + "id_to_date"
        if not os.path.exists(id_to_date_path):
            file_id_to_date = {}
        else:
            with open(id_to_date_path,"rb") as file :
                file_id_to_date = pickle.load(file)
        for call_id, date in id_to_date.items():
            file_id_to_date[call_id] = date

        with open(id_to_date_path,"wb") as file : 
            pickle.dump(file_id_to_date, file, protocol=pickle.HIGHEST_PROTOCOL)
        
        time.sleep(60)

  0%|          | 0/200 [00:00<?, ?it/s]

The new directory data/sp600/ticker/AAN has been created!


  0%|          | 0/57 [00:00<?, ?it/s]

The new directory data/sp600/ticker/AAON has been created!


  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

The new directory data/sp600/ticker/CUTR has been created!


  0%|          | 0/18 [00:00<?, ?it/s]

The new directory data/sp600/ticker/CVBF has been created!


KeyboardInterrupt: 

Download HTML files 

domain_name = 'https://seekingalpha.com'
for call_id, URL in tqdm(id_URLs):
    file_path = html_path + "/"+ call_id + ".html"
    with open(file_path, "w") as file :
        url = domain_name+URL
        html =  requests.get(url).text
        file.write(html)

Save the last scrapped url of the ticker, so that if bot detected can resume

with open(html_path+"/last_time_scrap", "w") as file:
    file.write(last_publish_time_min)

## Check last file got downloaded, i.e. captcha/bot error 

BeautifulSoup(html).get_text()[:2000]

Save date of transcripts

id_to_date_path = "data/ticker/id_to_date"
if not os.path.exists(id_to_date_path):
    file_id_to_date = {}
else:
    with open(id_to_date_path,"rb") as file :
        file_id_to_date = pickle.load(file)
for call_id, date in id_to_date.items():
    file_id_to_date[call_id] = date
    
with open(id_to_date_path,"wb") as file : 
    pickle.dump(file_id_to_date, file, protocol=pickle.HIGHEST_PROTOCOL)
    